# Fine-Tune Whisper

Adapted from guide here: https://huggingface.co/blog/fine-tune-whisper

## Load Dataset

In [1]:
from datasets import load_dataset

dataset = load_dataset("openpecha/stt-training-data", split='train').select(range(10_000))

dataset = dataset.train_test_split(.1)

dataset

DatasetDict({
    train: Dataset({
        features: ['file_name', 'uni', 'wylie', 'url', 'dept', 'grade', 'char_len', 'audio_len'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['file_name', 'uni', 'wylie', 'url', 'dept', 'grade', 'char_len', 'audio_len'],
        num_rows: 1000
    })
})

## Prepare Feature Extractor, Tokenizer and Data

In [2]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="bo", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="bo", task="transcribe")

### Prepare Data

In [3]:
import librosa
import requests
import tempfile
import numpy as np

def prepare_dataset(batch):
    # Download audio from URL
    response = requests.get(batch["url"])
    response.raise_for_status()

    # Save to temporary WAV file
    with tempfile.NamedTemporaryFile(suffix=".wav") as tmp:
        tmp.write(response.content)
        tmp.flush()

        # Load and resample audio using librosa
        waveform, sr = librosa.load(tmp.name, sr=16000)

    # Feature extraction
    batch["input_features"] = feature_extractor(
        waveform, sampling_rate=16000
    ).input_features[0]

    # Tokenize transcription
    batch["labels"] = tokenizer(
        batch["uni"], max_length=448, truncation=True
    ).input_ids

    return batch



In [4]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/9000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [5]:
dataset.save_to_disk('processed-ds')

Saving the dataset (0/18 shards):   0%|          | 0/9000 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

## Training and Evaluation

In [6]:
"""from datasets import load_from_disk

from transformers import WhisperProcessor
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer


feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")

dataset = load_from_disk('processed-ds')

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="bo", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="bo", task="transcribe")"""

'from datasets import load_from_disk\n\nfrom transformers import WhisperProcessor\nfrom transformers import WhisperFeatureExtractor\nfrom transformers import WhisperTokenizer\n\n\nfeature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")\n\ndataset = load_from_disk(\'processed-ds\')\n\ntokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="bo", task="transcribe")\nfeature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")\nprocessor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="bo", task="transcribe")'

In [7]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

In [8]:
model.generation_config.language = "bo"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

### Define a Data Collator

In [9]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [10]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### Evaluation Metrics

In [11]:
import evaluate
from tibetan_wer.wer import wer as tib_wer

cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")

Loading Trie... (0s.)


In [12]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    tib_wer_res = tib_wer(predictions=pred_str, references=label_str)

    macro_wer = tib_wer_res['macro_wer']
    micro_wer = tib_wer_res['micro_wer']
    subs = tib_wer_res['substitutions']
    ins = tib_wer_res['insertions']
    dels = tib_wer_res['deletions']


    return {"cer": cer,
            "standard_wer": wer,
            "tib_macro_wer": macro_wer,
            "tib_micro_wer": micro_wer,
            "substitutions": subs,
            "insertions": ins,
            "deletions": dels
            }

### Define the Training Configuration and Train

In [13]:
%env WAND_PROJECT=tib-wer-poc

env: WAND_PROJECT=tib-wer-poc


learning rate from: https://github.com/vasistalodagala/whisper-finetune

In [14]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="poc-run",  # change to a repo name of your choice
    auto_find_batch_size=True,
    #per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    learning_rate=3.75e-5, # tiny: 3.75e-5, base: 2.5e-5, tiny: 1.25e-5, med: 6.25e-6, large: 4.375e-6
    gradient_checkpointing=True,
    fp16=True,
    #per_device_eval_batch_size=2,
    predict_with_generate=True,
    generation_max_length=225,
    evaluation_strategy='epoch',
    #load_best_model_at_end=True,
    metric_for_best_model="cer",
    greater_is_better=False,
    num_train_epochs=5
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

trainer.train()

/home/j/Documents/MLotsawa/.venv/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_9506/2379653324.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/home/j/Documents/MLotsawa/.venv/lib/python3.12/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please ref

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Epoch,Training Loss,Validation Loss,Cer,Standard Wer,Tib Macro Wer,Tib Micro Wer,Substitutions,Insertions,Deletions
1,No log,0.559379,0.790729,1.000436,1.261686,1.061738,12321,3171,3924
2,0.633400,0.374469,0.605291,1.022698,0.902918,0.871767,10638,1508,3796
3,0.633400,0.313105,0.559658,1.015714,0.825856,0.815990,9655,1242,4025
4,0.303300,0.282869,0.533724,1.033610,0.784910,0.784000,9159,1233,3945


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/j/Documents/MLotsawa/.venv/lib/python3.12/site-packages/botok/textunits/bostring.py:82: UserWarning: Beware of unexpected results: input string contains the non-expanded char "བྷ", found in "".
  warn(
/home/j/Documents/MLotsawa/.venv/lib/python3.12/site-packages/botok/textunits/bostring.py:82: UserWarning: Beware of unexpected results: input string contains the non-expanded char "གྷ", found in "".
  warn(
/home/j/Documents/MLotsawa/.venv/lib/python3.12/site-packages/botok/textunits/bostring.py:82: UserWarning: Beware of unexpected results: input string con

TrainOutput(global_step=1405, training_loss=0.3991153866370802, metrics={'train_runtime': 5086.1962, 'train_samples_per_second': 8.847, 'train_steps_per_second': 0.276, 'total_flos': 1.10450166202368e+18, 'train_loss': 0.3991153866370802, 'epoch': 4.984888888888889})

In [15]:
model.save_pretrained('poc-tiny-fine-tuned')